In [21]:
import asyncio
import httpx


async def fetch_product(client: httpx.AsyncClient, url: str) -> dict:
    response = await client.get(url)
    return response.json()


async def scrape_shopify_products(base_url: str, client: httpx.AsyncClient) -> list:
    all_products = []
    page_number = 1

    while True:
        data = await fetch_product(
            client,
            f"{base_url}/products.json?page={page_number}"
        )

        products = data.get("products", [])
        if not products:
            break

        all_products.extend(products)
        page_number += 1

    return all_products


# Run scraping
async def main():
    async with httpx.AsyncClient() as client:
        products = await scrape_shopify_products("https://cf.com.lb", client)

    print(f"Found {len(products)} products 🛍️")

# Fix: Directly await main() when an event loop is already running (e.g., in Colab)
await main()

/usr/lib/python3.12/json/decoder.py:354: RuntimeWarning: coroutine 'main' was never awaited
  obj, end = self.scan_once(s, idx)


Found 4225 products 🛍️


The `products` variable now contains a list of `ShopifyProduct` objects. Let's inspect the first few products to see the structure of the data.

In [22]:
# Display the total number of products found
print(f"Total products scraped: {len(products)}")

# Display the first product in the list
if products:
    print("\nFirst product:")
    print(products[0].model_dump_json(indent=2))

# Optionally, display the title and price of the first variant of the first product
# if products and products[0].variants:
#     print("\nFirst product's first variant title and price:")
#     print(f"Title: {products[0].variants[0].title}")
#     print(f"Price: {products[0].variants[0].price}")

Total products scraped: 4225

First product:
{
  "id": 8788552188053,
  "title": "Ombres G Eyeshadow Quad",
  "vendor": "Guerlain",
  "tags": [
    "Eye",
    "Eyeshadow",
    "Luxury",
    "Make Up",
    "Make Up Palettes"
  ],
  "variants": [
    {
      "id": 46052464525461,
      "title": "011 Imperial Moon",
      "price": "109.00",
      "available": true,
      "sku": "I045870",
      "featured_image": {
        "id": 43338970136725,
        "src": "https://cdn.shopify.com/s/files/1/0716/5791/4517/files/3346470436558.jpg?v=1748355550",
        "width": 2048,
        "height": 2048
      }
    },
    {
      "id": 46052464558229,
      "title": "214 Exotic Orchid",
      "price": "109.00",
      "available": true,
      "sku": "I045871",
      "featured_image": {
        "id": 43338970202261,
        "src": "https://cdn.shopify.com/s/files/1/0716/5791/4517/files/3346470436510.jpg?v=1748355565",
        "width": 2048,
        "height": 2048
      }
    },
    {
      "id": 4605246

In [ ]:
import json

# Convert Pydantic models to dictionaries
products_dict_list = [product.model_dump() for product in products]

# Save the list of dictionaries to a JSON file
file_path = 'products.json'
with open(file_path, 'w') as f:
    json.dump(products_dict_list, f, indent=4)

print(f"Products data saved to {file_path}")

Products data saved to products.json


In [17]:
import json
import requests
from bs4 import BeautifulSoup

url = "https://cf.com.lb/recommendations/products?product_id=8984818778261&section_id=template--19295305793685__main"
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')


variant_script = soup.find("script", {"data-variant": True, "type": "application/json"})

if variant_script:
    variant_json_str = variant_script.string.strip()
    variant_data = json.loads(variant_json_str)
    print("Extracted Variant JSON:")
    print(json.dumps(variant_data, indent=2))
else:
    print("Variant JSON script not found")

Extracted Variant JSON:
{
  "id": 46696701165717,
  "title": "60ml",
  "option1": "60ml",
  "option2": null,
  "option3": null,
  "sku": "I056771",
  "requires_shipping": true,
  "taxable": true,
  "featured_image": {
    "id": 45046109372565,
    "product_id": 8984818778261,
    "position": 1,
    "created_at": "2025-12-04T17:34:59+03:00",
    "updated_at": "2025-12-04T17:34:59+03:00",
    "alt": null,
    "width": 2048,
    "height": 2048,
    "src": "//cf.com.lb/cdn/shop/files/3386460169080.jpg?v=1764858899",
    "variant_ids": [
      46696701165717
    ]
  },
  "available": true,
  "name": "Original Pour Femme - Eau de Parfum - 60ml",
  "public_title": "60ml",
  "options": [
    "60ml"
  ],
  "price": 7901,
  "weight": 0,
  "compare_at_price": null,
  "inventory_management": "shopify",
  "barcode": "3386460169080",
  "featured_media": {
    "alt": null,
    "id": 36408661246101,
    "position": 1,
    "preview_image": {
      "aspect_ratio": 1.0,
      "height": 2048,
      "width

In [19]:
def extract_variants_from_html(html: str):
    soup = BeautifulSoup(html, "html.parser")

    variants = []
    scripts = soup.find_all("script", {"type": "application/json", "data-variant": True})

    for script in scripts:
        try:
            json_data = json.loads(script.string.strip())
            variants.append(json_data)
        except Exception as e:
            print("Skipping script due to parsing error:", e)

    return variants


# Example usage
if __name__ == "__main__":
    html = """PASTE YOUR HTML HERE"""
    result = extract_variants_from_html(html)
    print(json.dumps(result, indent=2))

[
  {
    "id": 46696701165717,
    "title": "60ml",
    "option1": "60ml",
    "option2": null,
    "option3": null,
    "sku": "I056771",
    "requires_shipping": true,
    "taxable": true,
    "featured_image": {
      "id": 45046109372565,
      "product_id": 8984818778261,
      "position": 1,
      "created_at": "2025-12-04T17:34:59+03:00",
      "updated_at": "2025-12-04T17:34:59+03:00",
      "alt": null,
      "width": 2048,
      "height": 2048,
      "src": "//cf.com.lb/cdn/shop/files/3386460169080.jpg?v=1764858899",
      "variant_ids": [
        46696701165717
      ]
    },
    "available": true,
    "name": "Original Pour Femme - Eau de Parfum - 60ml",
    "public_title": "60ml",
    "options": [
      "60ml"
    ],
    "price": 7901,
    "weight": 0,
    "compare_at_price": null,
    "inventory_management": "shopify",
    "barcode": "3386460169080",
    "featured_media": {
      "alt": null,
      "id": 36408661246101,
      "position": 1,
      "preview_image": {
     

In [28]:
import json
from bs4 import BeautifulSoup

def clean_html(html_text: str) -> str:
    """Remove HTML tags and return clean text."""
    if not html_text:
        return ""
    return BeautifulSoup(html_text, "html.parser").get_text(separator=" ").strip()

def parse_shopify_products(data: dict):
    products = data.get("products", [])

    extracted_data = []

    for p in products:
        handle = p.get("handle")
        if handle:
            product_url = f"https://cf.com.lb/products/{handle}"
        else:
            product_url = None
        product_info = {
            "product_id": p.get("id"),
            "title": p.get("title"),
            "handle": product_url,
            "vendor": p.get("vendor"),
            "product_type": p.get("product_type"),
            "body_html_clean": clean_html(p.get("body_html", "")),
            "images": [img.get("src") for img in p.get("images", [])],
            "options": {
                "size": [],
                "color": []
            },
            "variants": []
        }

        # Extract size & color options
        for opt in p.get("options", []):
            name = opt.get("name", "").lower()
            if "size" in name:
                product_info["options"]["size"] = opt.get("values", [])
            if "color" in name:
                product_info["options"]["color"] = opt.get("values", [])

        # Variants
        for v in p.get("variants", []):
            variant_info = {
                "variant_id": v.get("id"),
                "title": v.get("title"),
                "sku": v.get("sku"),
                "available": v.get("available"),
                "price": v.get("price"),
                "grams": v.get("grams"),
            }
            product_info["variants"].append(variant_info)

        extracted_data.append(product_info)

    return extracted_data


# ========= Example Usage =========

# Load your JSON example
# json_str = """PUT_JSON_HERE"""
# shopify_data = json.loads(products)

# Fix: Pass a dictionary with a 'products' key containing the list of product dictionaries
parsed = parse_shopify_products({"products": products_dict_list})
parsed[0]
# for prod in parsed:
#     print(json.dumps(prod, indent=2))

/tmp/ipython-input-1066198500.py:37: RuntimeWarning: coroutine 'main' was never awaited
  for opt in p.get("options", []):


{'product_id': 8788552188053,
 'title': 'Ombres G Eyeshadow Quad',
 'handle': None,
 'vendor': 'Guerlain',
 'product_type': None,
 'body_html_clean': '',
 'images': [],
 'options': {'size': [], 'color': []},
 'variants': [{'variant_id': 46052464525461,
   'title': '011 Imperial Moon',
   'sku': 'I045870',
   'available': True,
   'price': '109.00',
   'grams': None},
  {'variant_id': 46052464558229,
   'title': '214 Exotic Orchid',
   'sku': 'I045871',
   'available': True,
   'price': '109.00',
   'grams': None},
  {'variant_id': 46052464459925,
   'title': '258 Wild Nudes',
   'sku': 'I049472',
   'available': True,
   'price': '109.00',
   'grams': None},
  {'variant_id': 46052464590997,
   'title': '360 Mystic Peacock',
   'sku': 'I045872',
   'available': True,
   'price': '109.00',
   'grams': None},
  {'variant_id': 46726528794773,
   'title': '520 Stellar Glow',
   'sku': 'I056842',
   'available': False,
   'price': '115.00',
   'grams': None},
  {'variant_id': 46052464623765,

In [31]:
import requests
from bs4 import BeautifulSoup
import json
import time

BASE_URL = "https://cf.com.lb"

def get_val(obj, key, default=None):
    if isinstance(obj, dict):
        return obj.get(key, default)
    return getattr(obj, key, default)


for product in products:
    product_id = get_val(product, "id")
    if not product_id:
        continue

    print(f"🔍 Getting barcode for product: {get_val(product,'title')} ({product_id})")

    url = f"{BASE_URL}/recommendations/products?product_id={product_id}&section_id=template--19295305793685__main"

    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")

        variant_script = soup.find("script", {"data-variant": True, "type": "application/json"})
        if not variant_script:
            print("  ❌ No variant script found")
            continue

        try:
            raw_data = json.loads(variant_script.text)

            # Normalize into list ALWAYS
            if isinstance(raw_data, dict):
                variant_json = [raw_data]
            elif isinstance(raw_data, list):
                variant_json = raw_data
            else:
                print("⚠ Unexpected JSON structure")
                continue

        except json.JSONDecodeError:
            print("⚠ Failed to decode JSON")
            continue

        for variant in get_val(product, "variants", []):
            v_id = get_val(variant, "id")

            match = next((v for v in variant_json
                          if isinstance(v, dict) and v.get("id") == v_id), None)

            barcode = match.get("barcode") if match else None

            # Save back into object
            if barcode:
                if isinstance(variant, dict):
                    variant["barcode"] = barcode
                else:
                    setattr(variant, "barcode", barcode)

                print(f"  ✔ {get_val(variant,'title')} → {barcode}")
            else:
                print(f"  ❌ {get_val(variant,'title')} barcode missing")


        time.sleep(1)

    except Exception as exc:
        print(f"Error: {exc}")

print("\n🎉 All barcodes processed!")


🔍 Getting barcode for product: Ombres G Eyeshadow Quad (8788552188053)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Dolce Vita - Eau de Toilette (8989277946005)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Dune Pour Homme - Eau de Toilette (8989276274837)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Eau Sauvage - Eau de Toilette (8989275062421)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Dior Homme After Shave Lotion (8989261627541)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Dior Homme Deodorant Stick (8989261398165)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Million Gold For Her Gift Set (8867229597845)
  ❌ No variant script found
🔍 Getting barcode for product: Invictus Victory Elixir Gift Set (8867229499541)
  ❌ No variant script found
🔍 Getti

KeyboardInterrupt: 

In [33]:
import asyncio
import httpx
import json
import requests
from bs4 import BeautifulSoup
import time

BASE_URL = "https://cf.com.lb"


# ----------------------------------------
# 🛍️ Fetch products from Shopify
# ----------------------------------------
async def fetch_page(client, url):
    res = await client.get(url)
    return res.json()


async def scrape_products():
    all_products = []
    page = 1

    async with httpx.AsyncClient() as client:
        while True:
            url = f"{BASE_URL}/products.json?page={page}"
            print(f"Fetching: {url}")

            data = await fetch_page(client, url)
            products = data.get("products", [])

            if not products:
                break

            all_products.extend(products)
            page += 1

    print(f"✔ Total products fetched: {len(all_products)}")
    return all_products


# ----------------------------------------
# 📦 Fetch barcode for each product variant
# ----------------------------------------
def add_barcodes(products):

    for p in products:
        product_id = p.get("id")
        if not product_id:
            continue

        print(f"\n🔍 {p.get('title')} ({product_id})")

        url = f"{BASE_URL}/recommendations/products?product_id={product_id}&section_id=template--19295305793685__main"

        try:
            html = requests.get(url).text
            soup = BeautifulSoup(html, "html.parser")

            script_tag = soup.find("script", {"data-variant": True, "type": "application/json"})
            if not script_tag:
                print("  ❌ No variant script")
                continue

            raw_json = json.loads(script_tag.text)

            # Normalize JSON to list
            variant_data = raw_json if isinstance(raw_json, list) else [raw_json]

            for v in p.get("variants", []):
                v_id = v.get("id")
                match = next((x for x in variant_data if x.get("id") == v_id), None)

                barcode = match.get("barcode") if match else None
                v["barcode"] = barcode

                if barcode:
                    print(f"  ✔ {v.get('title')} → {barcode}")
                else:
                    print(f"  ❌ {v.get('title')} no barcode")

        except Exception as e:
            print(f"  ⚠ Error: {e}")

        time.sleep(0.7)  # Shopify safety / avoid blocking

    print("\n🎉 Barcode scraping completed")


# ----------------------------------------
# 🚀 Run Sequence
# ----------------------------------------
# ----------------------------------------
# 🚀 Run Sequence on Google Colab
# ----------------------------------------
import nest_asyncio
nest_asyncio.apply()

loop = asyncio.get_event_loop()
products = loop.run_until_complete(scrape_products())

add_barcodes(products)


Streaming output truncated to the last 5000 lines.

🔍 Gentleman Intense - Eau de Toilette (8798186930325)
  ✔ 60ml → 3274872422995
  ❌ 100ml no barcode

🔍 Reserve Privee Gift Set (8798186471573)
  ❌ No variant script

🔍 Gentleman - Eau de Parfum Boisee (8798186274965)
  ✔ 60ml → 3274872425002
  ❌ 100ml no barcode

🔍 Gentleman - Eau de Toilette (8798186111125)
  ✔ 60ml → 3274872424999
  ❌ 100ml no barcode

🔍 Gentleman - Eau de Parfum (8798185947285)
  ✔ 50ml → 3274872424982
  ❌ 100ml no barcode

🔍 Gentleman - Eau de Cologne (8798185783445)
  ✔ 50ml → 3274872382374

🔍 Gentlman Hair and Body Shower Gel (8798185619605)
  ✔ 150ml → 3274872368811

🔍 Dahlia Divin Nude - Eau de Parfum (8798185422997)
  ✔ 50ml → 3274872350830

🔍 Dahlia Divin Eau Initial - Eau de Toilette (8798185193621)
  ✔ 50ml → 3274872365933

🔍 Pour Homme - Eau De Toilette (8796667510933)
  ✔ 100ml → 3274870303166

🔍 Le Parfum Royal - Eau de Parfum (8796753854613)
  ✔ 50ml → 7640233340080
  ❌ 90ml no barcode

🔍 Elie Saab L'H

In [34]:
import json

# Convert Pydantic models to dictionaries
# products_dict_list = [product.model_dump() for product in products]

# Save the list of dictionaries to a JSON file
file_path = 'products.json'
with open(file_path, 'w') as f:
    json.dump(products, f, indent=4)

print(f"Products data saved to {file_path}")

Products data saved to products.json


In [35]:
products[0]

{'id': 8788552188053,
 'title': 'Ombres G Eyeshadow Quad',
 'handle': 'ombres-g-eyeshadow-quad',
 'body_html': '<p>Ombres G eyeshadow quads have an exceptional texture made up of intense pigments for long-lasting colour, offering unparalleled depth.</p>\n\n<p>True gems of nature, the Ombres G adorn the eyes with four different finishes: an intense satin, a deep matte, a luminous metallic finish and a radiant pearl.</p>\n\n<p>Violette, Guerlain Creative Director of Make-Up, has imagined generous and balanced palettes, which allow you to play with shades, textures and finishes, creating a host of effects to endlessly transform your eyes both day and night.</p>',
 'published_at': '2025-12-08T17:34:57+03:00',
 'created_at': '2025-05-27T12:05:32+03:00',
 'updated_at': '2025-12-09T05:21:24+03:00',
 'vendor': 'Guerlain',
 'product_type': 'Luxury',
 'tags': ['Eye', 'Eyeshadow', 'Luxury', 'Make Up', 'Make Up Palettes'],
 'variants': [{'id': 46052464525461,
   'title': '011 Imperial Moon',
   'o